In [34]:
import pandas as pd
from config import api_key
import requests

In [63]:
url = f'https://api.pubg.com/shards/console/samples'
header= {"Authorization": f"Bearer {api_key}","Accept": "application/vnd.api+json"}
response = requests.get(url,headers=header)
r = response.json()
matches =r['data']['relationships']['matches']['data']
match_name = pd.json_normalize(matches)
match_name.to_csv('sample_matchIds.csv')

In [64]:
r

{'data': {'type': 'sample',
  'id': '340da9be-d354-44df-9f2b-e3b6bff6e7b4',
  'attributes': {'createdAt': '2021-08-13T00:00:00Z',
   'titleId': 'bluehole-pubg',
   'shardId': 'console'},
  'relationships': {'matches': {'data': [{'type': 'match',
      'id': 'b62ce27e-d6c8-47d7-bc0b-d7c16858074f'},
     {'type': 'match', 'id': '920cdf19-adec-4f0a-99e3-6e03dde7aa5c'},
     {'type': 'match', 'id': '02e8332c-55aa-417f-9615-8794342a0d9c'},
     {'type': 'match', 'id': 'cc15050c-9b83-4cfb-a575-2b328a66646c'},
     {'type': 'match', 'id': 'a75db296-5b70-4b36-8493-2b97f5543759'},
     {'type': 'match', 'id': 'f53be974-a518-405d-bb4e-680e72cd8f70'},
     {'type': 'match', 'id': '8d5a301d-f596-43d8-bba4-62494e260781'},
     {'type': 'match', 'id': '679797e2-62a7-4e37-a6fd-7318f99898b2'},
     {'type': 'match', 'id': '0338584e-8ac1-4c26-bc98-795c60b20612'},
     {'type': 'match', 'id': '24ff1f9c-9b2a-4409-b701-3248c3ffcd21'},
     {'type': 'match', 'id': '2de2ded1-e684-4312-b9b6-2bfdf8545d7b'},
 

In [81]:
duration_list = []
map_list =[]
game_mode = []
match_type = []
match_list = []
stats_list = []
asset_id = []
for match in match_name['id']:
    match_id = match
    url = f'https://api.pubg.com/shards/console/matches/{match_id}'
    header= {"Authorization": f"Bearer {api_key}","Accept": "application/vnd.api+json"}
    response = requests.get(url,headers=header)
    r = response.json()
    length = len(r['included'])-1
    for x in range(length):
        match_list.append(match_id)
        asset_id.append(r['data']['relationships']['assets']['data'][0]['id'])
        map_list.append(r['data']['attributes']['mapName'])
        duration_list.append(r['data']['attributes']['duration'])
        game_mode.append(r['data']['attributes']['gameMode'])
        match_type.append(r['data']['attributes']['matchType'])
        df =f'{x}'
        stats = r['included'][x]['attributes']
        df = pd.json_normalize(stats)
        stats_list.append(df)


In [82]:
new_df = pd.concat(stats_list)
new_df.drop(['actor', 'shardId','won', 'createdAt', 'URL', 'name', 'description',
             'stats.teamId'], axis=1, inplace=True)
new_df.insert(0, 'matchId', match_list)
new_df.insert(1, 'assetsId' , asset_id)
new_df.insert(2, 'gameDuration', duration_list)
new_df.insert(3, 'gameMode', game_mode)
new_df.insert(4, 'map', map_list)
new_df.insert(5, 'matchType', match_type)

In [83]:
new_df.set_index('stats.name', inplace = True)

In [84]:
new_df['stats.rank'].fillna(-1, inplace = True)
rank_list = []
for rank in new_df['stats.rank']:
    if rank == -1:
        rank_list.append('Unranked')
    else:
        rank_list.append(rank)
new_df['stats.rank'] = rank_list
new_df.dropna(inplace = True)

In [86]:
new_df.to_csv('match_samps.csv')